In [1]:
library(Seurat)
library(Signac)
library(CellChat)
library(patchwork)
library(argparse)
source("/tscc/projects/ps-epigen/users/kdang/eye_final/source_FunFig.R")
library(dplyr)                                                               
library(devtools)
library(hdf5r)
source("/tscc/projects/ps-epigen/bpToolbox/utils.R")

Loading required package: SeuratObject

Loading required package: sp




Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following object is masked from ‘package:Seurat’:

    components


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: ggplot2

Loading required package: usethis

Warning message:
“package ‘hdf5r’ was built under R version 4.4.2”


## Adding data JB_1495

In [2]:
#sobj.data <- Read10X_h5("/tscc/projects/ps-epigen/users/a2jorgensen/sandbox/250225_humanAT2_organoids/data/Cellranger/JB_1466/raw_feature_bc_matrix.h5")
lib1.data <- Read10X("/tscc/projects/ps-epigen/10x_output/250311/counts_out/snRNA/JB_1495/outs/raw_feature_bc_matrix/")
# Load Demuxlet best files
demux_lib1 <- read.table("/tscc/projects/ps-epigen/users/a2jorgensen/sandbox/250225_humanAT2_organoids/data/Demux/250317_AT2_organoid_ZO/popscle_out/JB_1495.best", header = TRUE, sep = "\t", stringsAsFactors = FALSE)

unique(colnames(demux_lib1))

[1] "INT_ID"             "BARCODE"            "NUM.SNPS"          
 [4] "NUM.READS"          "DROPLET.TYPE"       "BEST.GUESS"        
 [7] "BEST.LLK"           "NEXT.GUESS"         "NEXT.LLK"          
[10] "DIFF.LLK.BEST.NEXT" "BEST.POSTERIOR"     "SNG.POSTERIOR"     
[13] "SNG.BEST.GUESS"     "SNG.BEST.LLK"       "SNG.NEXT.GUESS"    
[16] "SNG.NEXT.LLK"       "SNG.ONLY.POSTERIOR" "DBL.BEST.GUESS"    
[19] "DBL.BEST.LLK"       "DIFF.LLK.SNG.DBL"

In [3]:
table(demux_lib1$DROPLET.TYPE)
rownames(demux_lib1) <- demux_lib1$BARCODE


  AMB   DBL   SNG 
29622   198 10213 

In [4]:
sobj.lib1 <- CreateSeuratObject(counts = lib1.data, project = "JB_1495_Organoid")

# Extract barcodes from Seurat object
seurat_barcodes <- rownames(sobj.lib1@meta.data)

# Extract barcodes from Demuxlet table
demux_barcodes <- rownames(demux_lib1)

In [5]:
# Print examples of barcodes
cat("Seurat Barcodes (first 10):\n")
print(head(seurat_barcodes, 10))

cat("Demuxlet Barcodes (first 10):\n")
print(head(demux_barcodes, 10))

Seurat Barcodes (first 10):
 [1] "AAACCAAAGAACCTAT-1" "AAACCAAAGAACCTTG-1" "AAACCAAAGAACGATA-1"
 [4] "AAACCAAAGAACGGGC-1" "AAACCAAAGAAGCAAC-1" "AAACCAAAGAAGCACG-1"
 [7] "AAACCAAAGAAGCCAG-1" "AAACCAAAGAAGCCCT-1" "AAACCAAAGAAGCTGG-1"
[10] "AAACCAAAGAAGGACT-1"
Demuxlet Barcodes (first 10):
 [1] "."                  "AAACCAAAGAAGGGCC-1" "AAACCAAAGCCCGTGA-1"
 [4] "AAACCAAAGCGCTCGA-1" "AAACCAAAGGAGCTAG-1" "AAACCAAAGGCTACCG-1"
 [7] "AAACCAAAGGTGGACA-1" "AAACCAAAGGTTAATC-1" "AAACCAAAGTATGAAG-1"
[10] "AAACCATTCACCATGA-1"


In [6]:
# Remove rows with "." from Demuxlet metadata
demux_lib1 <- demux_lib1[demux_barcodes != ".", , drop = FALSE]

# Re-extract cleaned Demuxlet barcodes
demux_barcodes <- rownames(demux_lib1)
# Find how many Demuxlet barcodes match Seurat barcodes
matching_barcodes <- sum(demux_barcodes %in% seurat_barcodes)
cat("Matching barcodes after removing '.':", matching_barcodes, "\n")

Matching barcodes after removing '.': 40032 


In [7]:
# Find barcodes in Demuxlet but not in Seurat
demux_not_in_seurat <- setdiff(demux_barcodes, seurat_barcodes)

# Find barcodes in Seurat but not in Demuxlet
seurat_not_in_demux <- setdiff(seurat_barcodes, demux_barcodes)

# Print differences
cat("Barcodes in Demuxlet but not in Seurat:\n")
print(head(demux_not_in_seurat, 10))  # Show first 10

cat("Barcodes in Seurat but not in Demuxlet:\n")
print(head(seurat_not_in_demux, 10))  # Show first 10

# Count differences
cat("Number of barcodes in Demuxlet but not in Seurat:", length(demux_not_in_seurat), "\n")
cat("Number of barcodes in Seurat but not in Demuxlet:", length(seurat_not_in_demux), "\n")


Barcodes in Demuxlet but not in Seurat:
character(0)
Barcodes in Seurat but not in Demuxlet:
 [1] "AAACCAAAGAACCTAT-1" "AAACCAAAGAACCTTG-1" "AAACCAAAGAACGATA-1"
 [4] "AAACCAAAGAACGGGC-1" "AAACCAAAGAAGCAAC-1" "AAACCAAAGAAGCACG-1"
 [7] "AAACCAAAGAAGCCAG-1" "AAACCAAAGAAGCCCT-1" "AAACCAAAGAAGCTGG-1"
[10] "AAACCAAAGAAGGACT-1"
Number of barcodes in Demuxlet but not in Seurat: 0 
Number of barcodes in Seurat but not in Demuxlet: 1400597 


In [8]:
# Keep only rows where barcodes match Seurat object
demux_lib1_subset <- demux_lib1[demux_barcodes %in% seurat_barcodes, , drop = FALSE]

# Verify barcode alignment
cat("Final number of matching barcodes:", nrow(demux_lib1_subset), "\n")


Final number of matching barcodes: 40032 


In [9]:
demux_barcodes <-rownames(demux_lib1_subset)

In [10]:
# Find barcodes in Demuxlet but not in Seurat
demux_not_in_seurat <- setdiff(demux_barcodes, seurat_barcodes)

# Find barcodes in Seurat but not in Demuxlet
seurat_not_in_demux <- setdiff(seurat_barcodes, demux_barcodes)

# Print differences
cat("Barcodes in Demuxlet but not in Seurat:\n")
print(head(demux_not_in_seurat, 10))  # Show first 10

cat("Barcodes in Seurat but not in Demuxlet:\n")
print(head(seurat_not_in_demux, 10))  # Show first 10

# Count differences
cat("Number of barcodes in Demuxlet but not in Seurat:", length(demux_not_in_seurat), "\n")
cat("Number of barcodes in Seurat but not in Demuxlet:", length(seurat_not_in_demux), "\n")

Barcodes in Demuxlet but not in Seurat:
character(0)
Barcodes in Seurat but not in Demuxlet:
 [1] "AAACCAAAGAACCTAT-1" "AAACCAAAGAACCTTG-1" "AAACCAAAGAACGATA-1"
 [4] "AAACCAAAGAACGGGC-1" "AAACCAAAGAAGCAAC-1" "AAACCAAAGAAGCACG-1"
 [7] "AAACCAAAGAAGCCAG-1" "AAACCAAAGAAGCCCT-1" "AAACCAAAGAAGCTGG-1"
[10] "AAACCAAAGAAGGACT-1"
Number of barcodes in Demuxlet but not in Seurat: 0 
Number of barcodes in Seurat but not in Demuxlet: 1400597 


In [11]:
seurat_barcodes<- as.data.frame(seurat_barcodes)
colnames(seurat_barcodes) <- "barcode"
head(seurat_barcodes)

,barcode
,<chr>
1,AAACCAAAGAACCTAT-1
2,AAACCAAAGAACCTTG-1
3,AAACCAAAGAACGATA-1
4,AAACCAAAGAACGGGC-1
5,AAACCAAAGAAGCAAC-1
6,AAACCAAAGAAGCACG-1


In [12]:
colnames(demux_lib1_subset)[colnames(demux_lib1_subset) == "BARCODE"] <- "barcode"


head(demux_lib1_subset)

,INT_ID,barcode,NUM.SNPS,NUM.READS,DROPLET.TYPE,BEST.GUESS,BEST.LLK,NEXT.GUESS,NEXT.LLK,DIFF.LLK.BEST.NEXT,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AAACCAAAGAAGGGCC-1,4,AAACCAAAGAAGGGCC-1,21,31,SNG,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-7.24,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-8.31,1.07,-9.7,1.00,0_0_3_ZBC3,-7.24,0_0_1_ZBC1,-12.74,0.00006,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-8.31,1.07
AAACCAAAGCCCGTGA-1,21,AAACCAAAGCCCGTGA-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.04,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.04,0.00,-3.0,0.82,0_0_1_ZBC1,-0.04,0_0_2_ZBC2,-0.04,0.06180,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.04,0.00
AAACCAAAGCGCTCGA-1,34,AAACCAAAGCGCTCGA-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.02,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.02,0.00,-3.1,0.77,0_0_1_ZBC1,-0.02,0_0_2_ZBC2,-0.02,0.05783,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.02,0.00
AAACCAAAGGAGCTAG-1,43,AAACCAAAGGAGCTAG-1,54,125,SNG,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-13.67,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-23.02,9.35,-16.0,1.00,0_0_2_ZBC2,-13.67,0_0_1_ZBC1,-49.64,0.00000,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-23.02,9.35
AAACCAAAGGCTACCG-1,57,AAACCAAAGGCTACCG-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.00,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
AAACCAAAGGTGGACA-1,67,AAACCAAAGGTGGACA-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.03,"0_0_5_ZBl2,0_0_5_ZBl2,0.00",-0.03,0.00,-3.1,0.79,0_0_1_ZBC1,-0.03,0_0_5_ZBl2,-0.03,0.05908,"0_0_1_ZBC1,0_0_5_ZBl2,0.50",-0.03,0.00


In [13]:
unique(colnames(sobj.lib1@meta.data))

[1] "orig.ident"   "nCount_RNA"   "nFeature_RNA"

In [14]:

# Join metadata into Seurat object using dplyr's left_join (ensuring alignment)
new_metadata <- sobj.lib1@meta.data %>%
  tibble::rownames_to_column(var = "barcode") %>%
  full_join(demux_lib1_subset, by = "barcode") %>%
  tibble::column_to_rownames(var = "barcode")

# Assign updated metadata back to Seurat object
sobj.lib1 <- AddMetaData(sobj.lib1, new_metadata)

# Check if metadata was added correctly
unique(colnames(sobj.lib1@meta.data))

[1] "orig.ident"         "nCount_RNA"         "nFeature_RNA"      
 [4] "INT_ID"             "NUM.SNPS"           "NUM.READS"         
 [7] "DROPLET.TYPE"       "BEST.GUESS"         "BEST.LLK"          
[10] "NEXT.GUESS"         "NEXT.LLK"           "DIFF.LLK.BEST.NEXT"
[13] "BEST.POSTERIOR"     "SNG.POSTERIOR"      "SNG.BEST.GUESS"    
[16] "SNG.BEST.LLK"       "SNG.NEXT.GUESS"     "SNG.NEXT.LLK"      
[19] "SNG.ONLY.POSTERIOR" "DBL.BEST.GUESS"     "DBL.BEST.LLK"      
[22] "DIFF.LLK.SNG.DBL"

In [15]:
sobj.lib1

An object of class Seurat 
33538 features across 1440629 samples within 1 assay 
Active assay: RNA (33538 features, 0 variable features)
 1 layer present: counts

In [16]:
table(sobj.lib1$DROPLET.TYPE)


  AMB   DBL   SNG 
29622   197 10213 

## Adding data JB_1496

In [17]:
lib2.data <-  Read10X("/tscc/projects/ps-epigen/10x_output/250311/counts_out/snRNA/JB_1496/outs/raw_feature_bc_matrix/")
demux_lib2 <- read.table("/tscc/projects/ps-epigen/users/a2jorgensen/sandbox/250225_humanAT2_organoids/data/Demux/250317_AT2_organoid_ZO/popscle_out/JB_1496.best", header = TRUE, sep = "\t", stringsAsFactors = FALSE)

In [18]:
table(demux_lib2$DROPLET.TYPE)


  AMB   DBL   SNG 
36206   181 10115 

In [19]:
head(demux_lib2)

,INT_ID,BARCODE,NUM.SNPS,NUM.READS,DROPLET.TYPE,BEST.GUESS,BEST.LLK,NEXT.GUESS,NEXT.LLK,DIFF.LLK.BEST.NEXT,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,0,.,6386,23237,DBL,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-3311.17,"0_0_3_ZBC3,0_0_1_ZBC1,0.50",-3311.17,0.00,0.0,1.00,0_0_1_ZBC1,-4223.35,0_0_3_ZBC3,-6005.15,0.00000,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-3311.17,-912.18
2,7,AAACCAAAGAAGGGTT-1,1,1,AMB,"0_0_6_ZBl3,0_0_6_ZBl3,0.00",-0.06,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-0.71,0.65,-2.9,0.85,0_0_6_ZBl3,-0.06,0_0_3_ZBC3,-0.71,0.06490,"0_0_3_ZBC3,0_0_6_ZBl3,0.50",-0.33,0.27
3,10,AAACCAAAGAATCGTT-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.00,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
4,37,AAACCAAAGCCCTTGC-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.02,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.02,0.00,-3.1,0.77,0_0_1_ZBC1,-0.02,0_0_2_ZBC2,-0.02,0.05783,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.02,0.00
5,56,AAACCAAAGCTTAGTG-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.00,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
6,73,AAACCAAAGGCTACCG-1,2,2,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.03,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-0.03,0.00,-3.1,0.78,0_0_1_ZBC1,-0.03,0_0_3_ZBC3,-0.03,0.05903,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-0.03,0.00


In [20]:
rownames(demux_lib2) <- demux_lib2$BARCODE

sobj.lib2 <- CreateSeuratObject(counts = lib2.data, project = "JB_1496_Organoid")

In [21]:
rownames(demux_lib2) <- demux_lib2$BARCODE
# Extract barcodes from Seurat object
seurat_barcodes <- rownames(sobj.lib2@meta.data)

# Extract barcodes from Demuxlet table
demux_barcodes <- rownames(demux_lib2)

# Print examples of barcodes
cat("Seurat Barcodes (first 10):\n")
print(head(seurat_barcodes, 10))

cat("Demuxlet Barcodes (first 10):\n")
print(head(demux_barcodes, 10))

Seurat Barcodes (first 10):
 [1] "AAACCAAAGAACCAGG-1" "AAACCAAAGAACCTAT-1" "AAACCAAAGAACCTCA-1"
 [4] "AAACCAAAGAACCTGC-1" "AAACCAAAGAACGGCA-1" "AAACCAAAGAACGGGC-1"
 [7] "AAACCAAAGAACGGTG-1" "AAACCAAAGAAGCAAC-1" "AAACCAAAGAAGCACG-1"
[10] "AAACCAAAGAAGCGAT-1"
Demuxlet Barcodes (first 10):
 [1] "."                  "AAACCAAAGAAGGGTT-1" "AAACCAAAGAATCGTT-1"
 [4] "AAACCAAAGCCCTTGC-1" "AAACCAAAGCTTAGTG-1" "AAACCAAAGGCTACCG-1"
 [7] "AAACCAAAGGCTCCAG-1" "AAACCAAAGGGTGAGC-1" "AAACCAAAGTGTCATG-1"
[10] "AAACCAAAGTTAGTAC-1"


In [22]:
# Find barcodes in Demuxlet but not in Seurat
demux_not_in_seurat <- setdiff(demux_barcodes, seurat_barcodes)

# Find barcodes in Seurat but not in Demuxlet
seurat_not_in_demux <- setdiff(seurat_barcodes, demux_barcodes)

# Print differences
cat("Barcodes in Demuxlet but not in Seurat:\n")
print(head(demux_not_in_seurat, 10))  # Show first 10

cat("Barcodes in Seurat but not in Demuxlet:\n")
print(head(seurat_not_in_demux, 10))  # Show first 10

# Count differences
cat("Number of barcodes in Demuxlet but not in Seurat:", length(demux_not_in_seurat), "\n")
cat("Number of barcodes in Seurat but not in Demuxlet:", length(seurat_not_in_demux), "\n")

Barcodes in Demuxlet but not in Seurat:
[1] "."
Barcodes in Seurat but not in Demuxlet:
 [1] "AAACCAAAGAACCAGG-1" "AAACCAAAGAACCTAT-1" "AAACCAAAGAACCTCA-1"
 [4] "AAACCAAAGAACCTGC-1" "AAACCAAAGAACGGCA-1" "AAACCAAAGAACGGGC-1"
 [7] "AAACCAAAGAACGGTG-1" "AAACCAAAGAAGCAAC-1" "AAACCAAAGAAGCACG-1"
[10] "AAACCAAAGAAGCGAT-1"
Number of barcodes in Demuxlet but not in Seurat: 1 
Number of barcodes in Seurat but not in Demuxlet: 1477353 


In [23]:
# Keep only rows where barcodes match Seurat object
demux_lib2_subset <- demux_lib2[demux_barcodes %in% seurat_barcodes, , drop = FALSE]

# Verify barcode alignment
cat("Final number of matching barcodes:", nrow(demux_lib2_subset), "\n")

Final number of matching barcodes: 46501 


In [24]:
demux_barcodes <-rownames(demux_lib2_subset)

# Find barcodes in Demuxlet but not in Seurat
demux_not_in_seurat <- setdiff(demux_barcodes, seurat_barcodes)

# Find barcodes in Seurat but not in Demuxlet
seurat_not_in_demux <- setdiff(seurat_barcodes, demux_barcodes)

# Print differences
cat("Barcodes in Demuxlet but not in Seurat:\n")
print(head(demux_not_in_seurat, 10))  # Show first 10

cat("Barcodes in Seurat but not in Demuxlet:\n")
print(head(seurat_not_in_demux, 10))  # Show first 10

# Count differences
cat("Number of barcodes in Demuxlet but not in Seurat:", length(demux_not_in_seurat), "\n")
cat("Number of barcodes in Seurat but not in Demuxlet:", length(seurat_not_in_demux), "\n")

Barcodes in Demuxlet but not in Seurat:
character(0)
Barcodes in Seurat but not in Demuxlet:
 [1] "AAACCAAAGAACCAGG-1" "AAACCAAAGAACCTAT-1" "AAACCAAAGAACCTCA-1"
 [4] "AAACCAAAGAACCTGC-1" "AAACCAAAGAACGGCA-1" "AAACCAAAGAACGGGC-1"
 [7] "AAACCAAAGAACGGTG-1" "AAACCAAAGAAGCAAC-1" "AAACCAAAGAAGCACG-1"
[10] "AAACCAAAGAAGCGAT-1"
Number of barcodes in Demuxlet but not in Seurat: 0 
Number of barcodes in Seurat but not in Demuxlet: 1477353 


In [25]:
seurat_barcodes<- as.data.frame(seurat_barcodes)
colnames(seurat_barcodes) <- "barcode"
head(seurat_barcodes)

,barcode
,<chr>
1,AAACCAAAGAACCAGG-1
2,AAACCAAAGAACCTAT-1
3,AAACCAAAGAACCTCA-1
4,AAACCAAAGAACCTGC-1
5,AAACCAAAGAACGGCA-1
6,AAACCAAAGAACGGGC-1


In [26]:
colnames(demux_lib2_subset)[colnames(demux_lib2_subset) == "BARCODE"] <- "barcode"


head(demux_lib2_subset)

,INT_ID,barcode,NUM.SNPS,NUM.READS,DROPLET.TYPE,BEST.GUESS,BEST.LLK,NEXT.GUESS,NEXT.LLK,DIFF.LLK.BEST.NEXT,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AAACCAAAGAAGGGTT-1,7,AAACCAAAGAAGGGTT-1,1,1,AMB,"0_0_6_ZBl3,0_0_6_ZBl3,0.00",-0.06,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-0.71,0.65,-2.9,0.85,0_0_6_ZBl3,-0.06,0_0_3_ZBC3,-0.71,0.06490,"0_0_3_ZBC3,0_0_6_ZBl3,0.50",-0.33,0.27
AAACCAAAGAATCGTT-1,10,AAACCAAAGAATCGTT-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.00,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
AAACCAAAGCCCTTGC-1,37,AAACCAAAGCCCTTGC-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.02,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.02,0.00,-3.1,0.77,0_0_1_ZBC1,-0.02,0_0_2_ZBC2,-0.02,0.05783,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.02,0.00
AAACCAAAGCTTAGTG-1,56,AAACCAAAGCTTAGTG-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.00,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
AAACCAAAGGCTACCG-1,73,AAACCAAAGGCTACCG-1,2,2,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.03,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-0.03,0.00,-3.1,0.78,0_0_1_ZBC1,-0.03,0_0_3_ZBC3,-0.03,0.05903,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-0.03,0.00
AAACCAAAGGCTCCAG-1,74,AAACCAAAGGCTCCAG-1,1,2,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.00,-3.1,0.77,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05724,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00


In [27]:
unique(colnames(sobj.lib2@meta.data))

[1] "orig.ident"   "nCount_RNA"   "nFeature_RNA"

In [28]:

# Join metadata into Seurat object using dplyr's left_join (ensuring alignment)
new_metadata <- sobj.lib2@meta.data %>%
  tibble::rownames_to_column(var = "barcode") %>%
  full_join(demux_lib2_subset, by = "barcode") %>%
  tibble::column_to_rownames(var = "barcode")

# Assign updated metadata back to Seurat object
sobj.lib2 <- AddMetaData(sobj.lib2, new_metadata)

# Check if metadata was added correctly
unique(colnames(sobj.lib2@meta.data))

[1] "orig.ident"         "nCount_RNA"         "nFeature_RNA"      
 [4] "INT_ID"             "NUM.SNPS"           "NUM.READS"         
 [7] "DROPLET.TYPE"       "BEST.GUESS"         "BEST.LLK"          
[10] "NEXT.GUESS"         "NEXT.LLK"           "DIFF.LLK.BEST.NEXT"
[13] "BEST.POSTERIOR"     "SNG.POSTERIOR"      "SNG.BEST.GUESS"    
[16] "SNG.BEST.LLK"       "SNG.NEXT.GUESS"     "SNG.NEXT.LLK"      
[19] "SNG.ONLY.POSTERIOR" "DBL.BEST.GUESS"     "DBL.BEST.LLK"      
[22] "DIFF.LLK.SNG.DBL"

## Add data JB_1466

In [29]:
lib3.data <-  Read10X("/tscc/projects/ps-epigen/10x_output/250214_IGM/counts_out/snRNA/JB_1466_1_2/outs/raw_feature_bc_matrix")
demux_lib3 <- read.table("/tscc/projects/ps-epigen/users/a2jorgensen/sandbox/250225_humanAT2_organoids/data/Demux/250317_AT2_organoid_ZO/popscle_out/JB_1466_1_2.best", header = TRUE, sep = "\t", stringsAsFactors = FALSE)

In [30]:
table(demux_lib3$DROPLET.TYPE)


  AMB   DBL   SNG 
35345   449 14505 

In [31]:
head(demux_lib3)

,INT_ID,BARCODE,NUM.SNPS,NUM.READS,DROPLET.TYPE,BEST.GUESS,BEST.LLK,NEXT.GUESS,NEXT.LLK,DIFF.LLK.BEST.NEXT,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,0,.,7430,36605,DBL,"0_0_2_ZBC2,0_0_3_ZBC3,0.50",-4079.04,"0_0_3_ZBC3,0_0_2_ZBC2,0.50",-4079.04,0,0.0,1.00,0_0_3_ZBC3,-6168.48,0_0_2_ZBC2,-8007.10,0.00000,"0_0_2_ZBC2,0_0_3_ZBC3,0.50",-4079.04,-2089.44
2,3,AAACCAAAGAAGCTAA-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
3,18,AAACCAAAGCATCAAT-1,1,1,AMB,"0_0_5_ZBl2,0_0_5_ZBl2,0.00",-0.06,"0_0_6_ZBl3,0_0_6_ZBl3,0.00",-0.06,0,-2.9,0.85,0_0_5_ZBl2,-0.06,0_0_6_ZBl3,-0.06,0.06490,"0_0_5_ZBl2,0_0_6_ZBl3,0.50",-0.06,0.00
4,20,AAACCAAAGCATCTCT-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
5,24,AAACCAAAGCCACAGT-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-0.01,0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_3_ZBC3,-0.01,0.05666,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-0.01,0.00
6,27,AAACCAAAGCCGCATG-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00


In [32]:
sobj.lib3 <- CreateSeuratObject(counts = lib3.data, project = "JB_1466_1_2_Organoid")

In [33]:
rownames(demux_lib3) <- demux_lib3$BARCODE
# Extract barcodes from Seurat object
seurat_barcodes <- rownames(sobj.lib3@meta.data)

# Extract barcodes from Demuxlet table
demux_barcodes <- rownames(demux_lib3)

# Print examples of barcodes
cat("Seurat Barcodes (first 10):\n")
print(head(seurat_barcodes, 10))

cat("Demuxlet Barcodes (first 10):\n")
print(head(demux_barcodes, 10))

Seurat Barcodes (first 10):
 [1] "AAACCAAAGAACCTCA-1" "AAACCAAAGAACCTGC-1" "AAACCAAAGAACCTTG-1"
 [4] "AAACCAAAGAACGAAC-1" "AAACCAAAGAACGATA-1" "AAACCAAAGAACGGAT-1"
 [7] "AAACCAAAGAAGCCAG-1" "AAACCAAAGAAGCCTC-1" "AAACCAAAGAAGCTAA-1"
[10] "AAACCAAAGAAGCTCC-1"
Demuxlet Barcodes (first 10):
 [1] "."                  "AAACCAAAGAAGCTAA-1" "AAACCAAAGCATCAAT-1"
 [4] "AAACCAAAGCATCTCT-1" "AAACCAAAGCCACAGT-1" "AAACCAAAGCCGCATG-1"
 [7] "AAACCAAAGCGATCCG-1" "AAACCAAAGCTAGTAT-1" "AAACCAAAGGAATGAC-1"
[10] "AAACCAAAGGAGCTTC-1"


In [34]:
# Find barcodes in Demuxlet but not in Seurat
demux_not_in_seurat <- setdiff(demux_barcodes, seurat_barcodes)

# Find barcodes in Seurat but not in Demuxlet
seurat_not_in_demux <- setdiff(seurat_barcodes, demux_barcodes)

# Print differences
cat("Barcodes in Demuxlet but not in Seurat:\n")
print(head(demux_not_in_seurat, 10))  # Show first 10

cat("Barcodes in Seurat but not in Demuxlet:\n")
print(head(seurat_not_in_demux, 10))  # Show first 10

# Count differences
cat("Number of barcodes in Demuxlet but not in Seurat:", length(demux_not_in_seurat), "\n")
cat("Number of barcodes in Seurat but not in Demuxlet:", length(seurat_not_in_demux), "\n")

Barcodes in Demuxlet but not in Seurat:
[1] "."
Barcodes in Seurat but not in Demuxlet:
 [1] "AAACCAAAGAACCTCA-1" "AAACCAAAGAACCTGC-1" "AAACCAAAGAACCTTG-1"
 [4] "AAACCAAAGAACGAAC-1" "AAACCAAAGAACGATA-1" "AAACCAAAGAACGGAT-1"
 [7] "AAACCAAAGAAGCCAG-1" "AAACCAAAGAAGCCTC-1" "AAACCAAAGAAGCTCC-1"
[10] "AAACCAAAGAAGGACT-1"
Number of barcodes in Demuxlet but not in Seurat: 1 
Number of barcodes in Seurat but not in Demuxlet: 1196859 


In [35]:
# Keep only rows where barcodes match Seurat object
demux_lib3_subset <- demux_lib3[demux_barcodes %in% seurat_barcodes, , drop = FALSE]

# Verify barcode alignment
cat("Final number of matching barcodes:", nrow(demux_lib3_subset), "\n")

Final number of matching barcodes: 50298 


In [36]:
demux_barcodes <-rownames(demux_lib3_subset)

In [37]:
# Find barcodes in Demuxlet but not in Seurat
demux_not_in_seurat <- setdiff(demux_barcodes, seurat_barcodes)

# Find barcodes in Seurat but not in Demuxlet
seurat_not_in_demux <- setdiff(seurat_barcodes, demux_barcodes)

# Print differences
cat("Barcodes in Demuxlet but not in Seurat:\n")
print(head(demux_not_in_seurat, 10))  # Show first 10

cat("Barcodes in Seurat but not in Demuxlet:\n")
print(head(seurat_not_in_demux, 10))  # Show first 10

# Count differences
cat("Number of barcodes in Demuxlet but not in Seurat:", length(demux_not_in_seurat), "\n")
cat("Number of barcodes in Seurat but not in Demuxlet:", length(seurat_not_in_demux), "\n")

Barcodes in Demuxlet but not in Seurat:
character(0)
Barcodes in Seurat but not in Demuxlet:
 [1] "AAACCAAAGAACCTCA-1" "AAACCAAAGAACCTGC-1" "AAACCAAAGAACCTTG-1"
 [4] "AAACCAAAGAACGAAC-1" "AAACCAAAGAACGATA-1" "AAACCAAAGAACGGAT-1"
 [7] "AAACCAAAGAAGCCAG-1" "AAACCAAAGAAGCCTC-1" "AAACCAAAGAAGCTCC-1"
[10] "AAACCAAAGAAGGACT-1"
Number of barcodes in Demuxlet but not in Seurat: 0 
Number of barcodes in Seurat but not in Demuxlet: 1196859 


In [38]:
seurat_barcodes<- as.data.frame(seurat_barcodes)
colnames(seurat_barcodes) <- "barcode"
head(seurat_barcodes)

,barcode
,<chr>
1,AAACCAAAGAACCTCA-1
2,AAACCAAAGAACCTGC-1
3,AAACCAAAGAACCTTG-1
4,AAACCAAAGAACGAAC-1
5,AAACCAAAGAACGATA-1
6,AAACCAAAGAACGGAT-1


In [39]:
colnames(demux_lib3_subset)[colnames(demux_lib3_subset) == "BARCODE"] <- "barcode"


head(demux_lib3_subset)

,INT_ID,barcode,NUM.SNPS,NUM.READS,DROPLET.TYPE,BEST.GUESS,BEST.LLK,NEXT.GUESS,NEXT.LLK,DIFF.LLK.BEST.NEXT,BEST.POSTERIOR,SNG.POSTERIOR,SNG.BEST.GUESS,SNG.BEST.LLK,SNG.NEXT.GUESS,SNG.NEXT.LLK,SNG.ONLY.POSTERIOR,DBL.BEST.GUESS,DBL.BEST.LLK,DIFF.LLK.SNG.DBL
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AAACCAAAGAAGCTAA-1,3,AAACCAAAGAAGCTAA-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
AAACCAAAGCATCAAT-1,18,AAACCAAAGCATCAAT-1,1,1,AMB,"0_0_5_ZBl2,0_0_5_ZBl2,0.00",-0.06,"0_0_6_ZBl3,0_0_6_ZBl3,0.00",-0.06,0.0,-2.9,0.85,0_0_5_ZBl2,-0.06,0_0_6_ZBl3,-0.06,0.06490,"0_0_5_ZBl2,0_0_6_ZBl3,0.50",-0.06,0.00
AAACCAAAGCATCTCT-1,20,AAACCAAAGCATCTCT-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
AAACCAAAGCCACAGT-1,24,AAACCAAAGCCACAGT-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-0.01,0.0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_3_ZBC3,-0.01,0.05666,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-0.01,0.00
AAACCAAAGCCGCATG-1,27,AAACCAAAGCCGCATG-1,1,1,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-0.01,"0_0_2_ZBC2,0_0_2_ZBC2,0.00",-0.01,0.0,-3.1,0.76,0_0_1_ZBC1,-0.01,0_0_2_ZBC2,-0.01,0.05666,"0_0_1_ZBC1,0_0_2_ZBC2,0.50",-0.01,0.00
AAACCAAAGCGATCCG-1,33,AAACCAAAGCGATCCG-1,29,64,AMB,"0_0_1_ZBC1,0_0_1_ZBC1,0.00",-10.13,"0_0_3_ZBC3,0_0_3_ZBC3,0.00",-10.22,0.1,-13.0,1.00,0_0_1_ZBC1,-10.13,0_0_3_ZBC3,-10.22,0.00000,"0_0_1_ZBC1,0_0_3_ZBC3,0.50",-8.78,-1.35


In [40]:
unique(colnames(sobj.lib3@meta.data))

[1] "orig.ident"   "nCount_RNA"   "nFeature_RNA"

In [41]:

# Join metadata into Seurat object using dplyr's left_join (ensuring alignment)
new_metadata <- sobj.lib3@meta.data %>%
  tibble::rownames_to_column(var = "barcode") %>%
  full_join(demux_lib3_subset, by = "barcode") %>%
  tibble::column_to_rownames(var = "barcode")

# Assign updated metadata back to Seurat object
sobj.lib3 <- AddMetaData(sobj.lib3, new_metadata)

# Check if metadata was added correctly
unique(colnames(sobj.lib3@meta.data))

[1] "orig.ident"         "nCount_RNA"         "nFeature_RNA"      
 [4] "INT_ID"             "NUM.SNPS"           "NUM.READS"         
 [7] "DROPLET.TYPE"       "BEST.GUESS"         "BEST.LLK"          
[10] "NEXT.GUESS"         "NEXT.LLK"           "DIFF.LLK.BEST.NEXT"
[13] "BEST.POSTERIOR"     "SNG.POSTERIOR"      "SNG.BEST.GUESS"    
[16] "SNG.BEST.LLK"       "SNG.NEXT.GUESS"     "SNG.NEXT.LLK"      
[19] "SNG.ONLY.POSTERIOR" "DBL.BEST.GUESS"     "DBL.BEST.LLK"      
[22] "DIFF.LLK.SNG.DBL"

In [42]:
table(sobj.lib3$SNG.BEST.GUESS)


0_0_1_ZBC1 0_0_2_ZBC2 0_0_3_ZBC3 0_0_4_ZBl1 0_0_5_ZBl2 0_0_6_ZBl3 
     28108       6429       9318       1648       3168       1627 

In [43]:
table(sobj.lib3$DROPLET.TYPE)


  AMB   DBL   SNG 
35345   448 14505 

In [44]:
table(sobj.lib2$SNG.BEST.GUESS)


0_0_1_ZBC1 0_0_2_ZBC2 0_0_3_ZBC3 0_0_4_ZBl1 0_0_5_ZBl2 0_0_6_ZBl3 
     32862       4685       1646       4207       1726       1375 

In [45]:
table(sobj.lib2$DROPLET.TYPE)


  AMB   DBL   SNG 
36206   180 10115 

In [46]:
table(sobj.lib1$SNG.BEST.GUESS)


0_0_1_ZBC1 0_0_2_ZBC2 0_0_3_ZBC3 0_0_4_ZBl1 0_0_5_ZBl2 0_0_6_ZBl3 
     22162       9601       2770       3367       1396        736 

In [47]:
table(sobj.lib1$DROPLET.TYPE)


  AMB   DBL   SNG 
29622   197 10213 

In [48]:
sobj <- merge(sobj.lib1, y = list(sobj.lib2, sobj.lib3), add.cell.ids = c("JB_1495", "JB_1496", "JB_1466_1_2"), project = "AT2_Organoid")

In [49]:
sobj

An object of class Seurat 
33538 features across 4211640 samples within 1 assay 
Active assay: RNA (33538 features, 0 variable features)
 3 layers present: counts.JB_1495_Organoid, counts.JB_1496_Organoid, counts.JB_1466_1_2_Organoid

In [50]:
table(sobj$DROPLET.TYPE)


   AMB    DBL    SNG 
101173    825  34833 

## Saving all lib objs

In [51]:
setwd("/tscc/projects/ps-epigen/users/a2jorgensen/sandbox/250225_humanAT2_organoids/data/rds_objs/proper_demux_add/")

In [52]:
saveRDS(sobj.lib1, "JB_1495_w_demux_data.RDS")

In [53]:
saveRDS(sobj.lib2, "JB_1496_w_demux_data.RDS")

In [54]:
saveRDS(sobj.lib3, "JB_1466_w_demux_data.RDS")

In [55]:
saveRDS(sobj, "250506_3_libs_w_demux.RDS")